In [1]:
import pandas as pd

In [2]:
# %load get_data.py
def get_data() :
    
    import pandas as pd
    
    csv_beer = pd.read_csv("/home/grimoire/Projects/BeerRatings/rating_update.csv")
    beer_ratings = pd.DataFrame(csv_beer)
    
    return beer_ratings


In [3]:
descriptions = pd.DataFrame(
    pd.read_csv('/home/grimoire/Projects/BeerRatings/beer_description.csv'))
ratings = get_data()

### This section will build a system that recommends beers styles that are similar to a particular beer style.
I will compute pairwise similarity scores for all beer styles based on their style descriptions and recommend beer styles based on that similarity score.

In [4]:
descriptions.keys()

Index(['style', 'description', 'abv_low', 'abv_high', 'ibu_low', 'ibu_high'], dtype='object')

In [5]:
# import TfidVectorizer from sklearn
'''
This computes Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each document. 
This will give a matrix where each column represents a word in the overview vocabulary 
(all the words that appear in at least one document) and each column represents a beer style, as before. 
'''
from sklearn.feature_extraction.text import TfidfVectorizer

# Remove all the stop words in the descriptions
tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(descriptions['description'])

In [6]:
tfidf_matrix.shape

(111, 1610)

In [7]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [8]:
indices = pd.Series(descriptions.index, index=descriptions['style']).drop_duplicates()

In [9]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    beer_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return descriptions['style'].iloc[beer_indices]

In [10]:
descriptions['style'].values

array(['German Bock', 'German Doppelbock', 'German Eisbock',
       'German Maibock', 'German Weizenbock', 'American Brown Ale',
       'English Brown Ale', 'English Dark Mild Ale', 'German Altbier',
       'American Black Ale', 'Belgian Dark Ale', 'Belgian Dubbel',
       'German Roggenbier', 'Scottish Ale', 'Winter Warmer',
       'American Amber / Red Lager', 'European Dark Lager',
       'German Märzen / Oktoberfest', 'German Rauchbier',
       'German Schwarzbier', 'Munich Dunkel Lager', 'Vienna Lager',
       'American Cream Ale', 'Bière de Champagne / Bière Brut', 'Braggot',
       'California Common / Steam Beer', 'American Brut IPA',
       'American Imperial IPA', 'American IPA', 'Belgian IPA',
       'English India Pale Ale (IPA)', 'New England IPA',
       'American Amber / Red Ale', 'American Blonde Ale',
       'American Pale Ale (APA)', 'Belgian Blonde Ale', 'Belgian Pale Ale',
       'Belgian Saison', 'English Bitter',
       'English Extra Special / Strong Bitter (ESB)

In [11]:
get_recommendations('American Brown Ale')

73                      Rye Beer
9             American Black Ale
108           Flanders Oud Bruin
47                American Lager
61                 Robust Porter
6              English Brown Ale
77         English Oatmeal Stout
56                German Pilsner
84     American Imperial Red Ale
35            Belgian Blonde Ale
Name: style, dtype: object

In [12]:
get_recommendations('British Barleywine')

83             American Barleywine
93              English Strong Ale
5               American Brown Ale
30    English India Pale Ale (IPA)
15      American Amber / Red Lager
1                German Doppelbock
75         American Imperial Stout
74                      Smoke Beer
65            Fruit and Field Beer
51    European Export / Dortmunder
Name: style, dtype: object

In [13]:
get_recommendations('American Wild Ale')

102                  American Brett
109                Flanders Red Ale
37                   Belgian Saison
92                  English Old Ale
30     English India Pale Ale (IPA)
64                    Finnish Sahti
107                  Belgian Lambic
29                      Belgian IPA
105            Belgian Fruit Lambic
98                  Berliner Weisse
Name: style, dtype: object

In [14]:
get_recommendations('Smoke Beer')

62                    Smoke Porter
18                German Rauchbier
51    European Export / Dortmunder
13                    Scottish Ale
4                German Weizenbock
3                   German Maibock
56                  German Pilsner
50               Bohemian Pilsener
1                German Doppelbock
21                    Vienna Lager
Name: style, dtype: object

In [15]:
# %load get_data.py
def get_data() :
    
    import pandas as pd
    
    csv_beer = pd.read_csv("/home/grimoire/Projects/BeerRatings/rating_update.csv")
    beer_ratings = pd.DataFrame(csv_beer)
    
    return beer_ratings


In [16]:
ratings = get_data()

In [17]:
beer_style_map = ratings.copy()

In [18]:
beer_style_map.keys()

Index(['brewery_id', 'brewery_name', 'review_time', 'review_overall',
       'review_aroma', 'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste', 'beer_name', 'beer_abv',
       'beer_beerid'],
      dtype='object')

In [19]:
beer_style_map = beer_style_map.drop(['brewery_id', 'brewery_name', 'review_time', 
                                      'review_profilename','beer_abv', 'beer_beerid'], axis=1)

In [20]:
comparison_beer = pd.DataFrame(pd.read_csv('/home/grimoire/Projects/BeerRatings/comparison_beer.csv'))

In [21]:
comparison_beer.keys()

Index(['beer_name', 'review_overall', 'review_taste', 'review_appearance',
       'review_palate', 'review_aroma', 'total_reviews'],
      dtype='object')

In [22]:
comparison_beer = comparison_beer.drop(['review_overall', 'review_taste', 'review_appearance',
                                        'review_palate', 'review_aroma'], axis=1)

In [23]:
beer_style_map = beer_style_map.merge(comparison_beer, left_on='beer_name', right_on='beer_name')

In [24]:
C = beer_style_map['review_overall'].mean()
m = 1 # 25th % Quartile
q_ratings = beer_style_map.copy().loc[beer_style_map['total_reviews'] >= m]

In [25]:
def weighted_rating(df, m=m, C=C):
    
    #########################################
    # Calculates and returns a weighted rating for specific feature
    # m is minimum votes/ratings required to be listed
    # C is the mean rating/vote across the whole dataframe
    # R is average rating/votes of feature
    # v is number of ratings/votes of feature
    #########################################
    
    v = df['total_reviews']
    R = df['review_overall']
    
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [26]:
q_ratings['weighted_overall'] = q_ratings.apply(weighted_rating, axis=1)

In [27]:
q_ratings = q_ratings.groupby(['beer_style', 'beer_name']).mean()

In [28]:
q_ratings.loc['Japanese Rice Lager', :]['weighted_overall'].sort_values(ascending=False).head(4)

beer_name
Mike Duggan #05 Asian Lager    4.528689
Atari Rice Ale                 4.157790
Heiwa                          4.157790
Samurai Gazebo                 4.119598
Name: weighted_overall, dtype: float64

In [29]:
def style_recommendation(title) :
    style_recommendations = []
    recommend = get_recommendations(title)
    recommend = recommend.reset_index(drop=True)
    for x in range(3) :
        style_recommendations.append(recommend.iloc[x])
    return style_recommendations

In [30]:
style_recommendation('American Adjunct Lager')

['Japanese Rice Lager', 'American Light Lager', 'European Strong Lager']

In [31]:
def beer_recommendation(df, style_list) :
    beer_temp = []
    for style in style_list :
        beer_temp.append(df.loc[style, :]['weighted_overall'].sort_values(ascending=False).head(4))
    beer_recommendations = pd.concat(beer_temp).sort_values(ascending=False)
    return beer_recommendations

In [32]:
results = beer_recommendation(q_ratings, style_recommendation('American Adjunct Lager'))
results

beer_name
Mike Duggan #05 Asian Lager        4.528689
Layflatter Lager                   4.438527
Kingfisher Super Strong            4.407790
Ohota Extra Light                  4.407790
Lemon Light                        4.407790
Willimantic Last Delivery Lager    4.407790
Bohemian Regent Kvasnicove 16°     4.157790
Harrington's Gold Lager            4.157790
Heiwa                              4.157790
Atari Rice Ale                     4.157790
Samurai Gazebo                     4.119598
Gordon Biersch Imperial Pilsner    4.105194
Name: weighted_overall, dtype: float64

In [34]:
style_recommendation('American Porter')

['American Brown Ale', 'Robust Porter', 'English Porter']

In [35]:
results = beer_recommendation(q_ratings, style_recommendation('American Porter'))
results

KeyError: 'the label [Robust Porter] is not in the [index]'

In [36]:
q_ratings.loc['Robust Porter', :]

KeyError: 'the label [Robust Porter] is not in the [index]'